# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:

import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.31.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-telco-churn'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException


cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.data.dataset_factory import DataType

#file uploaded to github
url = "https://raw.githubusercontent.com/thomascjw30/ML-Engineer-Capstone-Project/main/WA_Fn-UseC_-Telco-Customer-Churn.csv"

# pass url to Tabular dataset.  Note this is different to pandas dataframe, and gets converted to a dataframe in the function.

dataset = TabularDatasetFactory.from_delimited_files(url,header = True)

#Drop columns that are not needed 'customerID'
dataset = dataset.drop_columns(['customerID'])



In [5]:
ds = dataset.to_pandas_dataframe()
ds.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,True,False,1,False,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,Male,0,False,False,34,True,No,DSL,Yes,No,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,Male,0,False,False,2,True,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,Male,0,False,False,45,False,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,Female,0,False,False,2,True,No,Fiber optic,No,No,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True


In [6]:

# TODO: Put your automl config here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 10,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Churn",   
                             path = './telco_project',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#automl_config = AutoMLConfig()

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-telco-churn,AutoML_5f84fc51-a209-4bae-a867-b016ddf108ef,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
import azureml.train.automl

import azureml.automl.core
from azureml.automl.runtime.onnx_convert import OnnxConverter

remote_run.wait_for_completion()

metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

best_automl, best_model = remote_run.get_output()

Package:azureml-automl-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-core, training version:1.32.0, current version:1.31.0
Package:azureml-dataset-runtime, training version:1.32.0, current version:1.31.0
Package:azureml-defaults, training version:1.32.0, current version:1.31.0
Package:azureml-interpret, training version:1.32.0, current version:1.31.0
Package:azureml-mlflow, training version:1.32.0, current version:1.31.0
Package:azureml-pipeline-core, training version:1.32.0, current version:1.31.0
Package:azureml-telemetry, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-client, training version:1.32.0, current version:1.31.0
Package:azureml-train-automl-runtime, training version:1.32.0, current version:1.31.0


In [53]:
import joblib, pickle
joblib.dump(best_model, 'best_run_automl.pkl')

['best_run_automl.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [54]:
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, Webservice, AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import azureml.train.automl

model = Model.register(workspace = ws, model_name = 'best_run_automl', model_path = 'best_run_automl.pkl')
print(model.name, model.id, model.version, sep='\t')

# Create the environment
env = best_automl.get_environment()
#conda_dep = CondaDependencies()


Registering model best_run_automl
best_run_automl	best_run_automl:6	6


In [57]:
inference_config = InferenceConfig(entry_script='score.py', environment=env)

In [58]:

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
service = Model.deploy(ws, "telco-churn-automl-endpoint", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-28 15:10:30+00:00 Creating Container Registry if not exists.
2021-07-28 15:10:30+00:00 Registering the environment.
2021-07-28 15:10:31+00:00 Use the existing image.
2021-07-28 15:10:31+00:00 Generating deployment configuration.
2021-07-28 15:10:32+00:00 Submitting deployment to compute.
2021-07-28 15:10:35+00:00 Checking the status of deployment telco-churn-automl-endpoint..
2021-07-28 15:13:07+00:00 Checking the status of inference endpoint telco-churn-automl-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://bc43d9a7-3650-4f5f-a71e-f602b038012a.eastus2.azurecontainer.io/score
http://bc43d9a7-3650-4f5f-a71e-f602b038012a.eastus2.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [59]:
print("Scoring URI: " + service.scoring_uri)

Scoring URI: http://bc43d9a7-3650-4f5f-a71e-f602b038012a.eastus2.azurecontainer.io/score


In [60]:
import json
data = {"data":
        [
          {
            "gender" :"Female",
            "SeniorCitizen" :0,
            "Partner" :0,
            "Dependents" :0,
            "tenure" :1,
            "PhoneService" :0,
            "MultipleLines" :"No phone service",
            "InternetService" :"DSL",
            "OnlineSecurity" :0,
            "OnlineBackup" :0,
            "DeviceProtection" :0,
            "TechSupport" :0,
            "StreamingTV" :0,
            "StreamingMovies" :0,
            "Contract" :"Month-to-month",
            "PaperlessBilling" :1,
            "PaymentMethod" :"Electronic check",
            "MonthlyCharges" :1500.85,
            "TotalCharges" :1500.85,
          },
            
        {
            "gender" :"Male",
            "SeniorCitizen" :1,
            "Partner" :1,
            "Dependents" :0,
            "tenure" :0,
            "PhoneService" :0,
            "MultipleLines" :"No phone service",
            "InternetService" :"DSL",
            "OnlineSecurity" :0,
            "OnlineBackup" :0,
            "DeviceProtection" :0,
            "TechSupport" :0,
            "StreamingTV" :0,
            "StreamingMovies" :0,
            "Contract" :"Month-to-month",
            "PaperlessBilling" :0,
            "PaymentMethod" :"Electronic check",
            "MonthlyCharges" :20.50,
            "TotalCharges" :20.50,
            
        }
      ]}
test_sample = json.dumps(data)
output= service.run(test_sample)
print(output)

[False, False]


In [61]:
%run endpoint.py

[False, False]


TODO: In the cell below, print the logs of the web service and delete the service

In [62]:
print(service.get_logs())

2021-07-28T15:12:58,728877300+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-07-28T15:12:58,737153700+00:00 - rsyslog/run 
2021-07-28T15:12:58,745734200+00:00 - iot-server/run 
2021-07-28T15:12:58,785900500+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_fc92eee9a5613508afa12283dd0b27d8/lib/libuuid.so.1: no version information available (required by rsyslogd)
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (64)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 88
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-07-28T15:13:01,500229300+00:00 - iot-server/finish 1 0
2021-07-28T15:13:01,506411400+00:00 - Exit code 1 is normal. Not restarting iot-server.
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-07-28 15:13:03,150 | root | INFO | Starting up app insights client
logging socket was 

In [ ]:
service.delete()